## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor 

In [2]:
# 讀取資料集 (回歸問題)
boston = datasets.load_boston()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(boston.data, boston.target,
                                                    test_size=0.25, random_state=42)

# 建立模型
reg = GradientBoostingRegressor()

In [3]:
# 先看看使用預設參數得到的結果
reg = GradientBoostingRegressor(random_state=42)

# 訓練模型
reg.fit(x_train, y_train)

# 預測測試集
y_pred = reg.predict(x_test)
 
# 先看看使用預設參數得到的結果，約為 8.914 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

8.914847157043507


## [scoring 參數可參考 官方文件](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

In [4]:
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150]
max_depth = [3, 5, 7]
criterion = ["friedman_mse", "mse", "mae"]
min_samples_split=[2, 4]

param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion, min_samples_split=min_samples_split)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(reg, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1, cv=10, iid=True) 

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion -> 調整為10
# 總共 len(n_estimators)*len(max_depth)*len(criterion)*len(min_samples_split)=3*3*3*2=54 種參數組合
# 總共要 train 54*10(cv)=540 次模型

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 482 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   42.9s finished


In [5]:
# 印出最佳結果與最佳參數
print(f"Best Score: {grid_result.best_score_:.6f} using {grid_result.best_params_}")

Best Score: -11.176283 using {'criterion': 'mse', 'max_depth': 3, 'min_samples_split': 4, 'n_estimators': 150}


In [6]:
# 使用最佳參數重新建立模型
reg_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                          n_estimators=grid_result.best_params_['n_estimators'],
                                          criterion=grid_result.best_params_['criterion'],
                                          min_samples_split=grid_result.best_params_['min_samples_split'],
                                          random_state=42)

# 訓練模型
reg_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = reg_bestparam.predict(x_test)

# 調整參數後約可降至 8.711 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

8.711927928202607


### 注意 
雖然有做了cross-validation(cv參數預設3，此例調整至10)，提升模型泛化性。    
但是使用測試資料做評估，效果不見得會較好